# Assignment 1

## Decision Tree with no depth control -> Task 1A

### Load the data and codes

In [1]:
import pandas as pd
import numpy as np

init_df = pd.read_csv('data/training/adult_train_data.csv')
test_df = pd.read_csv('data/testing/adult_test_data.csv')

### Load the functions for best feature calculation based on information gain

In [2]:
"""
This module contains functions to calculate information gain for a given dataset.
"""


def map_features(df):
    """
    Map atrributes to their unique values.
    """
    column_feature_map = {}
    for column in df.columns:
        column_feature_map[column] = df[column].unique().tolist()
    return column_feature_map


def entropy(df):
    """
    Calculate the entropy of a label array.
    """
    counts = df['income_more50K'].value_counts()
    zero_count = counts.get(0, 0)
    one_count = counts.get(1, 0)
    total_count = zero_count + one_count
    if zero_count == 0 or one_count == 0:
        return 0
    # Adding a small constant to avoid log(0)
    entropy = -(
        (zero_count / total_count) * np.log2(zero_count / total_count + 1e-10) +
        (one_count / total_count) * np.log2(one_count / total_count + 1e-10)
    )
    return entropy


def information_gain(df, feature, target_column):
    """
    Calculate the information gain of a feature.
    """
    initial_entropy = entropy(df)

    weighted_entropy = 0
    correct_subset = df[df[target_column] == feature]
    incorrect_subset = df[df[target_column] != feature]
    total_count = len(df)
    correct_count = len(correct_subset)
    incorrect_count = len(incorrect_subset)

    correct_entropy = entropy(correct_subset)
    incorrect_entropy = entropy(incorrect_subset)

    weighted_entropy += (correct_count / total_count) * correct_entropy
    weighted_entropy += (incorrect_count / total_count) * incorrect_entropy

    return initial_entropy - weighted_entropy


def calculate_best_feature(df):
    """
    Find the best feature to split on.
    """
    column_feature_map = map_features(df)
    best_gain = -1
    best_feature = None
    best_feature_column = None

    for column, features in column_feature_map.items():
        if column == 'income_more50K':
            continue
        for feature in features:
            gain = information_gain(df, feature, column)
            if gain > best_gain:
                best_gain = gain
                best_feature = feature
                best_feature_column = column

    return best_feature, best_feature_column


### Load the TreeNode class for building decision tree

In [3]:
"""
This module contains the TreeNode class, which represents a node in a decision tree.
"""
class TreeNode:
    def __init__(self,
                 feature: str = None,
                 column: str = None,
                 data: pd.DataFrame = None,
                 label: str = None,
                 index: int = None,
                 max_depth: int = None,
                 left=None,
                 right=None):
        """
        Initialize a tree node with the given parameters.
        :param data: Data to train the node.
        :param feature: Feature to split on.
        :param column: Column name of the feature.
        :param label: Label of the node output
        value: Value of the node.
        """
        self.data = data
        self.feature = feature
        self.column = column
        self.label = label
        self.index = index
        self.max_depth = max_depth
        self.left = left
        self.right = right

    def __str__(self):
        return str(self.column) + ": " + str(self.feature)

    def train(self):
        """
        Train the tree node with the given data.
        :param data: Data to train the node.
        """
        # print(self.index)

        if self.data['income_more50K'].nunique() == 1:
            self.label = self.data['income_more50K'].values[0]
            return

        if self.max_depth is not None and self.index >= self.max_depth:
            self.label = self.data['income_more50K'].mode()[0]
            return

        self.feature, self.column = calculate_best_feature(self.data)
        right_data = self.data[self.data[self.column] == self.feature]
        left_data = self.data[self.data[self.column] != self.feature]

        if right_data.empty or left_data.empty:
            self.label = self.data['income_more50K'].mode()[0]
            return

        right_node = TreeNode(
            data=right_data, index=self.index + 1, max_depth=self.max_depth)
        left_node = TreeNode(
            data=left_data, index=self.index + 1, max_depth=self.max_depth)
        self.add_children(left_node, "left")
        self.add_children(right_node, "right")
        # Recursively train the left and right nodes

        left_node.train()
        right_node.train()

    def add_children(self, tree_node, direction: str):
        """
        Add children to the node.
        """

        if direction == "left":
            self.left = tree_node
        elif direction == "right":
            self.right = tree_node
        else:
            raise ValueError("Direction must be 'left' or 'right'")

    def print_tree(self, depth=0, direction=None):
        """ Recursively print the tree structure """

        if self.index > 6:
            return

        indent = "   " * depth
        if self.feature is not None:
            if direction == "left":
                print(
                    f"{indent}├── {self.index} No, then: check {self.column} to be {self.feature}")
            elif direction == "right":
                print(
                    f"{indent}├── {self.index} Yes, then: check {self.column} to be {self.feature}")
            else:
                print(
                    f"{indent}├── {self.index} Root: check {self.column} to be {self.feature}")
        else:
            if direction == "left":
                print(f"{indent}├── {self.index} No, then: output {self.label}")
            elif direction == "right":
                print(f"{indent}├── {self.index} Yes, then: output {self.label}")

        if self.right is not None:
            self.right.print_tree(depth=depth + 1, direction="right")
        if self.left is not None:
            self.left.print_tree(depth=depth + 1, direction="left")

    def predict(self, row):
        """
        Predict the label for the given row.
        """

        if self.label is not None:
            return int(self.label)

        if row[self.column] == self.feature:
            return self.right.predict(row)
        else:
            return self.left.predict(row)

    def predict_all(self, df):
        """
        Predict the labels for all rows in the given DataFrame.
        """
        predictions = []
        for _, row in df.iterrows():
            prediction = self.predict(row)
            predictions.append(prediction)
        return predictions

    def accuracy(self, df):
        """
        Calculate the accuracy of the decision tree on the given DataFrame.
        """
        predictions = self.predict_all(df)
        correct_predictions = sum(predictions == df['income_more50K'])
        accuracy = correct_predictions / len(df)
        return accuracy

    def recall(self, df):
        """
        Calculate the recall of the decision tree on the given DataFrame.
        """
        predictions = self.predict_all(df)
        true_positives = sum(
            (predictions == df['income_more50K']) & (df['income_more50K'] == 1))
        false_negatives = sum(
            (predictions != df['income_more50K']) & (df['income_more50K'] == 1))

        if true_positives + false_negatives == 0:
            return 0

        recall = true_positives / (true_positives + false_negatives)
        return recall

    def precision(self, df):
        """
        Calculate the precision of the decision tree on the given DataFrame.
        """
        predictions = self.predict_all(df)
        true_positives = sum(
            (predictions == df['income_more50K']) & (df['income_more50K'] == 1))
        false_positives = sum(
            (predictions != df['income_more50K']) & (df['income_more50K'] == 0))

        if true_positives + false_positives == 0:
            return 0

        precision = true_positives / (true_positives + false_positives)
        return precision

    def f1_score(self, df):
        """
        Calculate the F1 score of the decision tree on the given DataFrame.
        """
        recall = self.recall(df)
        precision = self.precision(df)
        if precision + recall == 0:
            return 0
        f1 = 2 * (precision * recall) / (precision + recall)
        return f1

    def confusion_matrix(self, df):
        """
        Calculate the confusion matrix of the decision tree on the given DataFrame.
        """

        predictions = self.predict_all(df)
        true_positives = sum(
            (predictions == df['income_more50K']) & (df['income_more50K'] == 1))
        false_positives = sum(
            (predictions != df['income_more50K']) & (df['income_more50K'] == 0))
        false_negatives = sum(
            (predictions != df['income_more50K']) & (df['income_more50K'] == 1))
        true_negatives = sum(
            (predictions == df['income_more50K']) & (df['income_more50K'] == 0))

        matrix = pd.DataFrame(
            [[true_positives, false_negatives], [false_positives, true_negatives]],
            index=["Actual Positive", "Actual Negative"],
            columns=["Predicted Positive", "Predicted Negative"]
        )

        return matrix

    def print_statistics(self, df):
        """
        Print the statistics of the decision tree on the given DataFrame.
        """
        print("")
        print("-------------------------------------------------")
        print("")
        print(f"Decision Tree Statistics for max depth {self.max_depth}:")
        print("")
        print(f"Training Accuracy: {self.accuracy(self.data) * 100:.2f}%")
        print(f"Testing Accuracy: {self.accuracy(df) * 100:.2f}%")
        print(f"Recall: {self.recall(df) * 100:.2f}%")
        print(f"Precision: {self.precision(df) * 100:.2f}%")
        print(f"F1 Score: {self.f1_score(df) * 100:.2f}%")

        print("Confusion Matrix:")
        print(self.confusion_matrix(df))


### Train the model and print output

In [4]:
decision_tree_no_max_depth = TreeNode(data=init_df, index=0)
decision_tree_no_max_depth.train()
decision_tree_no_max_depth.print_tree()

├── 0 Root: check marital_status to be Married-civ-spouse
   ├── 1 Yes, then: check education_num to be >=10
      ├── 2 Yes, then: check education to be Some-college
         ├── 3 Yes, then: check age to be 20-29
            ├── 4 Yes, then: check hours_per_week to be <35
               ├── 5 Yes, then: check fnlwgt to be 200K-300K
                  ├── 6 Yes, then: check occupation to be Adm-clerical
                  ├── 6 No, then: output 0
               ├── 5 No, then: check occupation to be Other-service
                  ├── 6 Yes, then: check fnlwgt to be 200K-300K
                  ├── 6 No, then: check occupation to be Machine-op-inspct
            ├── 4 No, then: check occupation to be Other-service
               ├── 5 Yes, then: check age to be 40-49
                  ├── 6 Yes, then: check relationship to be Own-child
                  ├── 6 No, then: check age to be >=60
               ├── 5 No, then: check occupation to be Farming-fishing
                  ├── 6 Yes, 

<h3>Task 1A</h3>
<h4>
    With no depth control, the training phase only stops if all labels at the node are the same and output the label or all data at the node have same attributes such that we cannot split using the criterion anymore.
</h4>
<h4>
    As the decision is too big with no depth control, I have only printed its first 6 levels. The condition at each check statement would be the feature that gives the best information gain when splitting at the node.
</h4>


## Decision Tree with depth control -> Task 1B

### Tree with max depth of 2

In [5]:
decision_tree_max_depth_2 = TreeNode(data=init_df, index=0, max_depth=2)
decision_tree_max_depth_2.train()
decision_tree_max_depth_2.print_tree()

├── 0 Root: check marital_status to be Married-civ-spouse
   ├── 1 Yes, then: check education_num to be >=10
      ├── 2 Yes, then: output 1
      ├── 2 No, then: output 0
   ├── 1 No, then: check hours_per_week to be >40
      ├── 2 Yes, then: output 0
      ├── 2 No, then: output 0


### Tree with max depth of 3

In [6]:
decision_tree_max_depth_3 = TreeNode(data=init_df, index=0, max_depth=3)
decision_tree_max_depth_3.train()
decision_tree_max_depth_3.print_tree()

├── 0 Root: check marital_status to be Married-civ-spouse
   ├── 1 Yes, then: check education_num to be >=10
      ├── 2 Yes, then: check education to be Some-college
         ├── 3 Yes, then: output 0
         ├── 3 No, then: output 1
      ├── 2 No, then: check education_num to be <8
         ├── 3 Yes, then: output 0
         ├── 3 No, then: output 0
   ├── 1 No, then: check hours_per_week to be >40
      ├── 2 Yes, then: check education_num to be >=10
         ├── 3 Yes, then: output 0
         ├── 3 No, then: output 0
      ├── 2 No, then: check occupation to be Prof-specialty
         ├── 3 Yes, then: output 0
         ├── 3 No, then: output 0


### Tree with max depth of 4

In [7]:
decision_tree_max_depth_4 = TreeNode(data=init_df, index=0, max_depth=4)
decision_tree_max_depth_4.train()
decision_tree_max_depth_4.print_tree()

├── 0 Root: check marital_status to be Married-civ-spouse
   ├── 1 Yes, then: check education_num to be >=10
      ├── 2 Yes, then: check education to be Some-college
         ├── 3 Yes, then: check age to be 20-29
            ├── 4 Yes, then: output 0
            ├── 4 No, then: output 0
         ├── 3 No, then: check education to be Assoc-voc
            ├── 4 Yes, then: output 0
            ├── 4 No, then: output 1
      ├── 2 No, then: check education_num to be <8
         ├── 3 Yes, then: check hours_per_week to be >40
            ├── 4 Yes, then: output 0
            ├── 4 No, then: output 0
         ├── 3 No, then: check age to be 20-29
            ├── 4 Yes, then: output 0
            ├── 4 No, then: output 0
   ├── 1 No, then: check hours_per_week to be >40
      ├── 2 Yes, then: check education_num to be >=10
         ├── 3 Yes, then: check age to be 20-29
            ├── 4 Yes, then: output 0
            ├── 4 No, then: output 0
         ├── 3 No, then: check age to be 20-29

<h3>Task 1B</h3>
<h4>
    Now with depth control, we just added another stopping criterion for the training phase. When the tree reaches the max depth, the node will immediately output the mode of the labels at the node, even if we can still split on the data. It is also seen that both directions of the node could output the same label as the prediction, and it becomes unnesscessary to check the last feature. This is because the labels have not been separated well, and 0 label is dominating the training dataset. Therefore, if both leaves output the mode, they would both predict 0.
</h4>


## Test Decision Tree with no depth control -> Task 1C

In [8]:
decision_tree_max_depth_3.print_statistics(test_df)


-------------------------------------------------

Decision Tree Statistics for max depth 3:

Training Accuracy: 81.29%
Testing Accuracy: 81.46%
Recall: 47.62%
Precision: 67.35%
F1 Score: 55.79%
Confusion Matrix:
                 Predicted Positive  Predicted Negative
Actual Positive                1762                1938
Actual Negative                 854               10506


<h3>Task 1C</h3>
<h4>
    In general, this decision tree fits the training data reasonably well, with a training accuracy of 82.29%. It also has a reasonably well performance on new data with a testing accuracy of 81.46%, which gives a very small variance. This indicates a very good generalisation of the model with a max depth of 3.
</h4>

<h4>
    More specifically, its recall is only 47.62%, while its precision is 67.35%. A low recall combined with a high precision suggests that the model is overly conservative in classifying positives, and it only predicts them when it is very confident. This is likely to be caused by the imbalance of the labels in the training dataset, where the negative examples dominate. When we output the mode at the node, the labels have not split very well at a depth of 3, and the dominating negative examples will cause the model to ignore the positive examples. This can be seen from the tree printed in 1B, that only one leaf outputs 1 while all others output 0.
</h4>


## Task 2

### Task 2A

<h4>
    If we change the splitting criterion to accuracy instead of information gain, the tree will prioritize splitting on the feature that maximizes prediction accuracy at each node. This could result in faster label separation at lower levels of the tree, potentially improving accuracy in the early stages. However, this approach ignores class distribution and the relationship between the label and other attributes, increasing the risk of overfitting. Since the model is not optimizing for information gain, it may focus on short-term accuracy rather than truly understanding how each feature contributes to the outcome, leading to a less generalizable model.
</h4>


### Task 2B

#### Print out the stats of each tree for comparison

In [9]:
decision_tree_max_depth_2.print_statistics(test_df)
decision_tree_max_depth_3.print_statistics(test_df)
decision_tree_max_depth_4.print_statistics(test_df)
decision_tree_no_max_depth.print_statistics(test_df)


-------------------------------------------------

Decision Tree Statistics for max depth 2:

Training Accuracy: 80.22%
Testing Accuracy: 80.66%
Recall: 63.41%
Precision: 60.09%
F1 Score: 61.70%
Confusion Matrix:
                 Predicted Positive  Predicted Negative
Actual Positive                2346                1354
Actual Negative                1558                9802

-------------------------------------------------

Decision Tree Statistics for max depth 3:

Training Accuracy: 81.29%
Testing Accuracy: 81.46%
Recall: 47.62%
Precision: 67.35%
F1 Score: 55.79%
Confusion Matrix:
                 Predicted Positive  Predicted Negative
Actual Positive                1762                1938
Actual Negative                 854               10506

-------------------------------------------------

Decision Tree Statistics for max depth 4:

Training Accuracy: 81.49%
Testing Accuracy: 81.70%
Recall: 43.89%
Precision: 70.49%
F1 Score: 54.10%
Confusion Matrix:
                 Predi

<h4>
    The tree with no depth control exhibits high variance, as indicated by the large gap between training and testing accuracy. This suggests that the model is overfitting the training data due to its complexity, capturing noise rather than generalizable patterns.
</h4>

<h4>
    Trees with a max depth of 3 and 4 show good generalization, as the difference between training and testing accuracy is minimal. However, their precision and recall are imbalanced, indicating that they are too strict in classifying positive examples. This could suggest underfitting, as the model struggles to effectively separate positive from negative examples. Given the imbalance in the training data (more negative examples), the model tends to classify more instances as negative.
</h4>

<h4>
    The tree with a maximum depth of 2 performs reasonably well overall. However, when compared to the deeper trees, testing accuracy tends to increase as complexity grows. This suggests that the model may benefit from greater depth, implying that it is also underfitting at this level.
</h4>